# <b>Spaceship Titanic</b>
Predict which passengers are transported to an alternate dimension.<br>
### <b>Description</b>
Welcome to the year 2912, where your data science skills are needed to solve a cosmic mystery. We've received a transmission from four lightyears away and things aren't looking good.

The Spaceship Titanic was an interstellar passenger liner launched a month ago. With almost 13,000 passengers on board, the vessel set out on its maiden voyage transporting emigrants from our solar system to three newly habitable exoplanets orbiting nearby stars.

While rounding Alpha Centauri en route to its first destination—the torrid 55 Cancri E—the unwary Spaceship Titanic collided with a spacetime anomaly hidden within a dust cloud. Sadly, it met a similar fate as its namesake from 1000 years before. Though the ship stayed intact, almost half of the passengers were transported to an alternate dimension!



To help rescue crews and retrieve the lost passengers, you are challenged to predict which passengers were transported by the anomaly using records recovered from the spaceship’s damaged computer system.

Help save them and change history!

## <b>Import Libraries</b>


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## <b>Defining Constants</b>

In [2]:
TRAIN_INPUT_PATH = './data/train.csv'
TEST_INPUT_PATH = './data/test.csv'

## <b>DATA EXPLORATION</b>
- train.csv - Personal records for about two-thirds (~8700) of the passengers, to be used as training data.  
    - **PassengerId**: A unique Id for each passenger. Each Id takes the form `gggg_pp` where `gggg` indicates a group the passenger is travelling with and `pp` is their number within the group. People in a group are often family members, but not always.  
    - **HomePlanet**: The planet the passenger departed from, typically their planet of permanent residence.  
    - **CryoSleep**: Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.  
    - **Cabin**: The cabin number where the passenger is staying. Takes the form `deck/num/side`, where `side` can be either `P` for Port or `S` for Starboard.  
    - **Destination**: The planet the passenger will be debarking to.  
    - **Age**: The age of the passenger.  
    - **VIP**: Whether the passenger has paid for special VIP service during the voyage.  
    - **RoomService, FoodCourt, ShoppingMall, Spa, VRDeck**: Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.  
    - **Name**: The first and last names of the passenger.  
    - **Transported**: Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.  

- test.csv - Personal records for the remaining one-third (~4300) of the passengers, to be used as test data. Your task is to predict the value of **Transported** for the passengers in this set.  

In [ ]:
df = pd.read_csv(TRAIN_INPUT_PATH, header=0, index_col=0)

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
PassengerId,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True


Missing values present in the CSV files.

In [1]:
import numpy as np

def atrous_convolution(image, kernel, rate=1, padding=0):
    """
    Esegui la convoluzione atrous su un'immagine con un determinato kernel, tasso di dilatazione e padding.
    
    :param image: Immagine di input (array NumPy 2D).
    :param kernel: Filtro/kernel da applicare (array NumPy 2D).
    :param rate: Tasso di dilatazione (default è 1, corrispondente a una convoluzione standard).
    :param padding: Numero di pixel di padding da aggiungere ai bordi dell'immagine (default è 0).
    :return: Immagine risultante dopo la convoluzione atrous.
    """
    kernel_height, kernel_width = kernel.shape
    img_height, img_width = image.shape

    # Calcola le dimensioni dell'output
    output_height = (img_height + 2 * padding - kernel_height + rate) // rate
    output_width = (img_width + 2 * padding - kernel_width + rate) // rate
    output = np.zeros((output_height, output_width))

    # Applica il padding all'immagine
    if padding > 0:
        image_padded = np.zeros((img_height + 2 * padding, img_width + 2 * padding))
        image_padded[padding:-padding, padding:-padding] = image
    else:
        image_padded = image

    # Applica la convoluzione atrous
    for i in range(output_height):
        for j in range(output_width):
            # Calcola la posizione corrispondente nell'immagine con padding
            y = i * rate
            x = j * rate
            # Assicurati che l'area del kernel sia all'interno dei limiti dell'immagine
            if y + kernel_height <= image_padded.shape[0] and x + kernel_width <= image_padded.shape[1]:
                output[i, j] = np.sum(image_padded[y:y + kernel_height, x:x + kernel_width] * kernel)
    return output


In [3]:
atrous_convolution(np.array([[1,2,3,4], [5, 6, 7,8], [9, 10, 11, 12], [13, 14, 15, 16]]), np.array([[1,1], [1,1]]), rate=1, padding=1).astype(int)

array([[ 1,  3,  5,  7,  4],
       [ 6, 14, 18, 22, 12],
       [14, 30, 34, 38, 20],
       [22, 46, 50, 54, 28],
       [13, 27, 29, 31, 16]])